# 在千帆 Python SDK 使用 OpenCompass 提供的功能

OpenCompass是由上海人工智能实验室开源的大模型评测平台。它涵盖了学科、语言、知识、理解、推理等五大评测维度，能够全面评估大模型的能力。OpenCompass作为一个评测工具，对于研究和开发大模型的人员来说，是非常有价值的资源。通过使用OpenCompass，用户可以更准确地了解他们的大模型在各项任务上的表现，从而进行针对性的优化和改进。

千帆 Python SDK 中内置的评估模块，支持用户使用 OpenCompass 提供的评估器和 `TEXT_POSTPROCESSORS` 对象，对模型的推理结果进行评估前的预处理，以及评估。

# 前置准备

首先，需要安装千帆 Python SDK

In [1]:
pip install -U qianfan

然后再安装 OpenCompass。这部分的教程可以参考 OpenCompass 所提供的[官方文档](https://opencompass.org.cn/doc)，或者直接使用以下命令在 Python 中安装：

In [2]:
pip install opencompass

# 正文

为了在千帆 Python SDK 的评估模块中使用来自 OpenCompass 的评估器，用户需要使用 `qianfan.evaluation.opencompass_evaluator` 中提供的 `OpenCompassLocalEvaluator` 类，将 OpenCompass 评估器包装为千帆 Python SDK 的评估器。

OpenCompass 所有可以使用的评估器都存放在 `opencompass.openicl.icl_evaluator` 模块下。其中，仅有，仅包含了 `predictions` 与 `references` 两个参数的 `score` 函数的评估器可用于千帆 Python SDK 提供的 `OpenCompassLocalEvaluator` 类。这部分符合条件的评估器包括：

+ opencompass.openicl.icl_evaluator.icl_agent_evaluator.PassRateEvaluator
+ opencompass.openicl.icl_evaluator.icl_circular_evaluator.CircularEvaluator
+ opencompass.openicl.icl_evaluator.icl_em_evaluator.EMEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.AccEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.RougeEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.BleuEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.BleuFloresEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.MccEvaluator
+ opencompass.openicl.icl_evaluator.icl_hf_evaluator.SquadEvaluator
+ opencompass.openicl.icl_evaluator.icl_jieba_rouge_evaluator.JiebaRougeEvaluator
+ opencompass.openicl.icl_evaluator.icl_toxic_evaluator.ToxicEvaluator

在本教程编写时，用户应可以直接使用上面列表所列的评估器。

而为了使用来自 OpenCompass 的 `TEXT_POSTPROCESSORS` 模块，用户应该首先对其进行一层封装：大多数 `TEXT_POSTPROCESSORS` 函数都只支持传入一个 `str` 参数作为入参。我们需要对 `TEXT_POSTPROCESSORS` 模块进行包装，使其能够额外接收 `**kwargs` 作为入参。

例如，手动编写一个 `wrapper` 函数

In [ ]:
def open_compass_text_post_processor_wrapper(fn: Callable) -> Callable:
    def wrapper(*args, **kwargs) -> str:
        return fn(*args)

    return wrapper

用户可以根据自身的需求，调整实际传入给 `TEXT_POSTPROCESSORS` 参数的入参。例如，有些 `TEXT_POSTPROCESSORS` 参数需要传入 `option` 参数。

下面我们演示如何使用其中的 `PassRateEvaluator` 评估器，`BleuEvaluator` 评估器和 `JiebaRougeEvaluator` 评估器来进行一次评估。

## 导入数据集

在评估之前，我们需要先导入数据集。本教程准备了一份专门用于评估的数据集，存放在 `data_file/eval_dataset.csv` 中

In [3]:
from qianfan.dataset import Dataset

ds = Dataset.load(data_file="data_file/eval_dataset.csv", eval_input_column="input_prompt", eval_llm_output_column="llm_output", reference_column="expected_output")

print(ds.list(0))

[INFO] [03-20 15:57:10] dataset.py:389 [t:8094817088]: no data source was provided, construct
[INFO] [03-20 15:57:10] dataset.py:257 [t:8094817088]: construct a file data source from path: data_file/eval_dataset.csv, with args: {'eval_input_column': 'input_prompt', 'eval_llm_output_column': 'llm_output', 'reference_column': 'expected_output'}
[INFO] [03-20 15:57:10] file.py:260 [t:8094817088]: use format type FormatType.Csv
[INFO] [03-20 15:57:10] utils.py:140 [t:8094817088]: has got a memory-mapped table
[INFO] [03-20 15:57:10] dataset.py:883 [t:8094817088]: list local dataset data by 0


{'prompt': '地球的自转周期是多久？', 'input_prompt': '地球的自转周期是多久？', 'llm_output': '地球的自转周期约为**23小时56分4秒**。', 'expected_output': '大约24小时', 'request_complete_latency': '0.0003000829999564303'}


# 创建评估器及评估器对象

然后，我们创建评估器对象。

In [4]:
from opencompass.openicl.icl_evaluator import PassRateEvaluator, JiebaRougeEvaluator, BleuEvaluator
from qianfan.evaluation import EvaluationManager
from qianfan.evaluation.opencompass_evaluator import OpenCompassLocalEvaluator

evaluator_list = [
    OpenCompassLocalEvaluator(open_compass_evaluator=JiebaRougeEvaluator()),
    OpenCompassLocalEvaluator(open_compass_evaluator=PassRateEvaluator()),
    OpenCompassLocalEvaluator(open_compass_evaluator=BleuEvaluator()),
]

em = EvaluationManager(local_evaluators=evaluator_list)

# 开始评估以及保存结果

我们只需要调用 `EvaluationManager` 的 `eval_only` 函数，即可完成一次评估，并且将评估结果保存到本地的文件中

In [5]:
result_ds = em.eval_only(ds).result_dataset

result_ds.save(data_file="eval_result.json")

[INFO] [03-20 15:57:13] dataset.py:883 [t:8094817088]: list local dataset data by 0
Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3g/cvyfl_zn12df2m4_1tj7mhdw0000gn/T/jieba.cache
Loading model cost 0.282 seconds.
Prefix dict has been built successfully.
[INFO] [03-20 15:58:11] dataset.py:462 [t:8094817088]: no destination data source was provided, construct
[INFO] [03-20 15:58:11] dataset.py:257 [t:8094817088]: construct a file data source from path: eval_result.json, with args: {}
[INFO] [03-20 15:58:11] file.py:260 [t:8094817088]: use format type FormatType.Json
[INFO] [03-20 15:58:11] dataset.py:883 [t:8094817088]: list local dataset data by slice(0, 9999, None)
